### 임베딩 객체 생성(쿼리 임베딩)

In [ ]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch

embedding_api_key = ""
embedding_endpoint = ""
embedding_api_version = "2024-02-15-preview"
embedding_deployment = "text-embedding-3-small"

os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)

ai_search_endpoint = ""
ai_search_api_key = ""

# ✅ 임베딩 객체 (쿼리를 임베딩하는 데 사용)
embedding = AzureOpenAIEmbeddings(
    api_key = embedding_api_key,
    azure_endpoint = embedding_endpoint,
    model = embedding_deployment,  
    openai_api_version = embedding_api_version
)

### source값 가져오기

In [3]:
import requests


def get_available_sources(k: int = 1000) -> list:
    """
    Azure AI Search 인덱스에서 사용 가능한 source 값 목록을 중복 없이 반환
    """
    

    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_api_key
    }

    body = {
        "search": "*",  # 전체 문서 검색
        "select": "source",  # source 필드만 가져옴
        "top": k
    }

    response = requests.post(ai_search_endpoint, headers=headers, json=body)

    if response.status_code != 200:
        print(f"❌ 요청 실패: {response.status_code}")
        print(response.text)
        return []

    data = response.json()
    sources = sorted(set(doc["source"] for doc in data["value"] if "source" in doc))
    return sources

In [4]:
sources = get_available_sources()
print(sources)


['(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf', '(정정공고문)25년1차청년매입임대_표준입주자모집공고문.pdf', '2025년 1차 대구경북 청년매입임대 입주자 모집 공고문.pdf', '2025년1차청년매입임대입주자모집공고문(광주전남).pdf', '25년 1차 청년매입임대 입주자 모집 공고문(강원지역본부).pdf', '25년1차청년매입임대입주자모집공고문.pdf', '아츠스테이영등포_입주자모집공고문.pdf']


In [5]:

def request_ai_search(query: str, source_filter: str = None, k: int = 5) -> list:
    # Azure AI Search 설정, endpoint에서 인덱스 지정정
    

    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_api_key
    }

    # 쿼리 임베딩
    query_vector = embedding.embed_query(query)

    # 요청 본문 구성
    body = {
        "vectorQueries": [
            {
                "kind": "vector",
                "vector": query_vector,
                "fields": "embedding",
                "k": k
            }
        ]
    }

    if source_filter:
        body["filter"] = f"source eq '{source_filter}'"

    # 요청 전송
    response = requests.post(ai_search_endpoint, headers=headers, json=body)

    # 응답 처리
    if response.status_code != 200:
        print(f"❌ 검색 실패: {response.status_code}")
        print(response.text)
        return []

    response_json = response.json()
    results = [
        {
            "content": item["content"],
            "source": item.get("source", ""),
            "score": item.get("@search.score", 0)
        }
        for item in response_json["value"]
    ]

    return results


In [6]:
query = "신청 기간이 어떻게 되나요?"
source = "(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf"

results = request_ai_search(query, source_filter=source, k=5)
for i, result in enumerate(results):
    print(f"\n🔍 Top {i+1} (score: {result['score']:.4f})")
    print(f"출처: {result['source']}")
    print(result["content"])



🔍 Top 1 (score: 0.6543)
출처: (대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf
|||신청자는 6개월, 당첨(예비)자는 5년, 계약자는 영구|
|전자우편서비스를 통한 임대료 고지서 및 각종 안내문 발송 임차인 중복계약(입주) 및 불법 양도·전대 확인 주택도시기금 대출 검색|||

🔍 Top 2 (score: 0.6304)
출처: (대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf
|공급단계|일정|내용|
|---|---|---|
|청약신청|4.7(월) 10:00 ~ 4.9(수) 16:00|• 청년 매입임대는 인터넷(https://apply.lh.or.kr) 또는 모바일 앱(LH청약플러스)으로 신청 가능합니다. * LH청약플러스→청약→임대주택→청약신청→매입임대/전세임대 • 공고내용을 숙지하신 후 신청자격, 소득 등 입력사항에 대하여 본인이 해당서류를 확인하시고 신청하시기 바랍니다. / 전화번호 등 개인정보 정확히 기재 요망 * 국세청 홈택스에서 소득조회 가능 (위임장 첨부 시 가족의 소득조회 가능) • 청약신청 기간 중 24시간 신청이 가능합니다. (단, 시작일과 마감일 제외) • 청약 시에는, 공동인증서(개인용) 또는 민간인증서(금융인증서, 네이버인증서, 토스인 증서, KB국민인증서) 발급을 신청접수 이전까지 미리 완료하여야 합니다.|

🔍 Top 3 (score: 0.6293)
출처: (대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf
취소됨을 안내받았습니다. 이에 따라 복학(입학) 후 1개월 이내에 재학증

 명서를 제출할 것임을 서약하며, 해당 기한 내에 제출하지 않을 경우에는

 당첨 및 계약이 취소되어도 일체의 이의를 제기하지 않을 것임을 확약합

 니다.

 20 년  월  일

 확인자 : (인)

### 한국토지주택공사  대전충남지역본부장 귀하


-----

## 근로형태 확인서

###### (19세 이상 39세 이하에 해당하지 않는 취업준비생의 직장 재직 여부 확인용)

 사업체명 법인등록번호
 (

### LLM 연동

In [ ]:
import requests
import re

# Azure OpenAI GPT-4o 설정
llm_endpoint = ""
llm_api_key = ""

def request_gpt(prompt: str) -> str:
    """
    GPT-4o에 프롬프트를 보내고 응답 받기
    """
    headers = {
        'Content-Type': 'application/json',
        'api-key': llm_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 친절하고 정확한 AI 도우미야. 사용자 질문에 문서 기반으로 답해줘."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(llm_endpoint, headers=headers, json=body)

    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        # [doc1] 같은 포맷이 있을 경우 사람이 읽기 쉽게 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content
    else:
        print("❌ 요청 실패:", response.status_code, response.text)
        return "⚠️ 오류가 발생했습니다."


In [8]:
def generate_answer_with_rag(query: str, source_filter: str = None, top_k: int = 3) -> str:
    # 🔍 벡터 유사도 기반 검색 수행 (REST API 기반)
    results = request_ai_search(query, source_filter=source_filter, k=top_k)

    # 결과가 없으면 안내
    if not results:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # 🔗 context 구성
    context = "\n\n".join([f"[doc{i+1}]\n{item['content']}" for i, item in enumerate(results)])

    # 🤖 LLM 프롬프트 생성
    prompt = f"""다음은 사용자가 질문한 내용과 관련된 문서 내용이야. 이 문서를 참고해서 질문에 대해 정확하고 구체적으로 답변해줘.
                그리고 어떤 문서에서 찾았는지도 출처도 알려줘
[사용자 질문]
{query}

[참고 문서]
{context}

답변:"""

    return request_gpt(prompt)


In [9]:
query = "내가 곧 결혼을 할 예정인데 혜택이 있을까요?"
source = "(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf"

answer = generate_answer_with_rag(query, source_filter=source, top_k=5)
print("🧠 GPT 응답:\n", answer)


🧠 GPT 응답:
 결혼을 하실 예정이라면, 임대주택 관련하여 혜택이 있을 수 있습니다. [참조 1]에 따르면, 입주 후 혼인한 경우 임대주택의 재계약이 추가로 연장될 수 있습니다. 일반적으로는 임대기간이 2년이며, 재계약을 최대 4회까지 가능해 최장 10년 거주할 수 있지만, 혼인한 경우 재계약이 5회 추가로 연장되어 최장 20년까지 거주할 수 있는 혜택을 받을 수 있습니다. 

이는 혼인 상태가 자격 조건을 충족하는 경우에만 해당하며, 시중 시세의 40~50% 수준의 임대료 및 임대보증금으로 입주할 수 있어 경제적 부담을 덜 수 있는 장점이 있습니다. 

출처: [참조 1]


In [10]:
print(sources)

['(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf', '(정정공고문)25년1차청년매입임대_표준입주자모집공고문.pdf', '2025년 1차 대구경북 청년매입임대 입주자 모집 공고문.pdf', '2025년1차청년매입임대입주자모집공고문(광주전남).pdf', '25년 1차 청년매입임대 입주자 모집 공고문(강원지역본부).pdf', '25년1차청년매입임대입주자모집공고문.pdf', '아츠스테이영등포_입주자모집공고문.pdf']
